# Intro

{Fill in with information about this notebook}

# Set Up notebook 

In [1]:
#Import modules
import pandas as pd
import csv
import numpy as np

#The rest of this is just for fun/quick dataviz at the end to map and ensure seems about right
import cartopy
import matplotlib.pyplot as plt
import rasterio
import geopandas

from lib import readData
from lib import mapping
from lib import cleanData
from lib import classify
from lib import exportData

todayDate, dateSuffix = readData.getCurrentDate()

# Read in data

- Set up filepaths
- Read in data from:
    - downholeData table (from database)
    - headerData table (from database)
    - xyzData file (from previously carried out work) (will eventually make this updateable)

Read in data

In [ ]:
downholeDataPATH, headerDataPATH, xyzInPATH  = readData.filesSetup()

#Functions to read data into dataframes. Also excludes extraneous columns
downholeDataIN, headerDataIN = readData.readRawTxtData(downholefile=downholeDataPATH, headerfile=headerDataPATH)
downholeDataIN, headerDataIN = readData.essentialCols(downholeDataIN, headerDataIN)
xyzDataIN = readData.readXYZData(xyzfile=xyzInPATH)

Define datatypes (doing this during the read in process has presented issues)

In [ ]:
#Define datatypes, to read into defineDataTypes() function
##EVENTUALLY, MAKE THIS A FILE IN THE RES FOLDER TO READ IN
downholeDataDTYPES = {'ID':np.uint32, "API_NUMBER":np.uint64,"TABLE_NAME":str,"WHO":str,"INTERPRET_DATE":str,"FORMATION":str,"THICKNESS":np.float64,"TOP":np.float64,"BOTTOM":np.float64}
headerDataDTYPES = {'ID':np.uint32,'API_NUMBER':np.uint64,"TDFORMATION":str,"PRODFORM":str,"TOTAL_DEPTH":np.float64,"SECTION":np.float64,"TWP":np.float64,"TDIR":str,"RNG":np.float64,"RDIR":str,"MERIDIAN":np.float64,"FARM_NAME":str,"NSFOOT":np.float64,"NSDIR":str,"EWFOOT":np.float64,"EWDIR":str,"QUARTERS":str,"ELEVATION":np.float64,"ELEVREF":str,"COMP_DATE":str,"STATUS":str,"FARM_NUM":str,"COUNTY_CODE":np.float64,"PERMIT_NUMBER":str,"COMPANY_NAME":str,"COMPANY_CODE":str,"PERMIT_DATE":str,"CORNER":str,"LATITUDE":np.float64,"LONGITUDE":np.float64,"ENTERED_BY":str,"UPDDATE":str,"ELEVSOURCE":str, "ELEV_FT":np.float64}
xyzDataDTYPES = {'ID':np.uint64, 'API_NUMBER':np.uint64, "LATITUDE":np.float64, "LONGITUDE":np.float64, "ELEV_FT":np.float64}

#Define datatypes of each column of the new dataframes
downholeDataIN = readData.defineDataTypes(downholeDataIN, js)
headerDataIN = readData.defineDataTypes(headerDataIN, headerDataDTYPES)
xyzDataIN = readData.defineDataTypes(xyzDataIN, xyzDataDTYPES)

#Make a copy of the data so raw data is preserved while we work with the rest of the data
downholeData = downholeDataIN.copy()
headerData = headerDataIN.copy()
xyzData = xyzDataIN.copy()

In [80]:
import json
downholeDataDTYPES = {'ID':np.uint32, "API_NUMBER":np.uint64,"TABLE_NAME":str,"WHO":str,"INTERPRET_DATE":str,"FORMATION":str,"THICKNESS":np.float64,"TOP":np.float64,"BOTTOM":np.float64}

dir =r'\\isgs-sinkhole\geophysics\Balikian\ISWS_HydroGeo\WellDataAutoClassification\SampleData\\'[:-1]
#dir = '../res/'
file = 'downholeDataTypes.txt'


with open(dir+file, 'r') as f:
    data= (f.read())
js = json.loads(data)

downholeDataPATH = r"\\isgs-sinkhole\geophysics\Balikian\ISWS_HydroGeo\RawWellData_OracleDatabase\TxtData\lcl_ISGS_DOWNHOLE_DATA.txt"
headerDataPATH = r"\\isgs-sinkhole\geophysics\Balikian\ISWS_HydroGeo\RawWellData_OracleDatabase\TxtData\lcl_ISGS_HEADER.txt"
downholeDataIN, headerDataIN = readData.readRawTxtData(rawdir='',downholefile=downholeDataPATH, headerfile=headerDataPATH)
downholeDataIN, headerDataIN = readData.essentialCols(downholeDataIN, headerDataIN)
downholeDataIN = readData.defineDataTypes(downholeDataIN, js)


Downhole Data has 3017085 valid well records.
Header Data has 634755 unique wells with valid location information.


AttributeError: module 'lib.readData' has no attribute 'essentialCols'

# Extract Elevation Data

Extract elevation data from consistent elevation dataset for all wells (lidar or other statewide DEM)

In [ ]:
#First, get wells with updated xyz info
    #Check first if xyzData needs to be updated with locations (?)
    #Check which wells in headerData don't have associated lidar data

#statewideLidar =  ow
#mapping.rastertoPoints_extract()

Merge elevation data with headerData table

In [ ]:
uniqueWells = headerData['API_NUMBER'].unique()
xyzData['UniqueWells'] = uniqueWells

headerData = mapping.addElevtoHeader(xyzData, headerData)
##NEED TO UPDATE THIS TO WORK WITH DATA WITH NO XYZ ELEVATION DATA FROM LIDAR
#Change xyz column name to indicate lidar
#Use order of preference: lidar, headerData table?/30/10m DEM?

# Data Cleaning

## First, let's clean up records in the data without the necessary information

HeaderXYZ dataframe has headerdata with updated elevations (from lidar or other DEM)

Now, remove data from downholeData table that does not have location information (Since we would not know where to put it anyway)

In [ ]:
downholeData = mapping.removeNonlocatedData(downholeData, headerData)

11440 records removed without Lat/Lon


Clip data from outside Illinois

In [ ]:
#Not done yet, use geopandas clip (May need to generate geometry column first in headerData)

Remove headerData rows without surface elevation information (this currently clips data from outside Illinois)

In [ ]:
headerData = cleanData.removenotopo(headerData, printouts=True)

Number of rows before dropping those without surface elevation information: 634755
Number of rows after dropping those without surface elevation information: 634533
Well records deleted: 222


Remove rows from downholeData with no depth information and where depth information is obviously bad (i.e., top depth > bottom depth)

In [ ]:
#Drop records with no depth information
donwholeData = cleanData.dropnodepth(downholeData, printouts=True)
#Drop records with bad depth information (i.e., top depth > bottom depth) (Also calculates thickness of each record)
donwholeData = cleanData.dropbaddepth(downholeData, printouts=True)

Drop records with no FORMATION information

In [ ]:
downholeData = cleanData.dropnoformation(downholeData, printouts=True)

Now we are going to export this data, to have record of cleaned data

In [ ]:
downholeData.reset_index(inplace=True,drop=True)
headerData.reset_index(inplace=True,drop=True)

downholeData.to_csv('../out/downholeData_cleaned'+dateSuffix+'.csv',index_label='ID')
headerData.to_csv('../out/headerData_cleaned'+dateSuffix+'.csv',index_label='ID')

# Classification

The following flags are used to mark the classification method:
- 0: Not classified
- 1: Specific Search Term Match
- 2: Wildcard match (startTerm) - no context
- 3: Bedrock classification for obvious bedrock
- 4: Wildcard match (startTerm) - with context

In [ ]:
#Read in dictionary files for downhole data
specTermsPATH, startTermsPATH = readData.searchTermFilePaths()

In [ ]:
specTerms, startTerms = readData.readSearchTerms(specfile=specTermsPATH, startfile=startTermsPATH)

Join the dataframes--for the specific search terms, this is the same as classifying them

In [ ]:
downholeData_spec = classify.specificDefine(downholeData, specTerms, printouts=True)
downholeData = downholeData_spec.copy()

Create a dataframe with only the records already classified (using the specific search terms in this case, classifiedDF), and one that still needs to be searched (searchDF)

In [ ]:
classifedDF, searchDF = classify.splitDefined(downholeData)
searchDF.shape[0]

Now, do the classification routine on the searchDF database

In [ ]:
searchDF = classify.startDefine(df=searchDF, starterms=startTerms, printouts=True)

Merge specDF and searchDF back together all back in single dataframe

In [ ]:
downholeData_Terms = classify.remergeData(classifieddf=classifedDF, searchdf=searchDF):
downholeData = downholeData_Terms.copy()

Export terms that still need to be defined to csv (along with their counts)

In [ ]:
classify.export_toBeDefined(downholeData, '../out/')

Classify all  data under depth threshold (default is 550') as bedrock (should not be an issue, but just in case)

In [ ]:
classifedDF, searchDF = classify.splitDefined(downholeData)
searchDF = classify.depthDefine(searchDF, thresh=550, printouts=True)
downholeData_Class = classify.remergeData(classifieddf=classifedDF, searchdf=searchDF)
downholeData = downholeData_Class.copy()

Add '0' flag for data still not classified

In [ ]:
downholeData = classify.fillUnclassified(downholeData)

In [ ]:
downholeData['CLASS_FLAG'].value_counts()

1.0    1161561
0.0     986327
3.0     273243
4.0     167146
Name: CLASS_FLAG, dtype: int64

## Add "Flag" for target interpratations

In [ ]:
#dictDir = "\\\\isgs-sinkhole\\geophysics\\Balikian\\ISWS_HydroGeo\\WellDataAutoClassification\\SupportingDocs\\"
InterpFile = 'Lithology_Interp_FineCoarse.csv'
targetInterpDF = pd.read_csv(dictDir+InterpFile)
targetInterpDF.rename(columns={'LITHOLOGY':'INTERPRETATION', 'CODE':'TARGET'}, inplace=True)
targetInterpDF

,INTERPRETATION,TARGET
0,BEDROCK,DoNotUse
1,FIRECLAY,DoNotUse
2,VOID,DoNotUse
3,CLAY AND STONE,0
4,DIRT AND BEDROCK,DoNotUse
5,DIRT AND STONE,DoNotUse
6,GRAVEL AND STONE,1
7,BOULDER,1
8,BOULDERS AND CLAY,0
9,GRAVEL WITH BOULDERS,1


In [ ]:
downholeData_targ = pd.merge(downholeData, targetInterpDF.set_index('INTERPRETATION'), right_on='INTERPRETATION',left_on='INTERPRETATION', how='left')
downholeData = downholeData_targ.copy()
downholeData['TARGET'].replace('DoNotUse', value=-1, inplace=True)
downholeData['TARGET'].fillna(value=-2, inplace=True)
downholeData['TARGET'].astype(np.int8)
downholeData

,API_NUMBER,TABLE_NAME,FORMATION,THICKNESS,TOP,BOTTOM,INTERPRETATION,CLASS_FLAG,TARGET
0,120010000300,WFORMATIONS,"Colchester No. 2 coal,Penn. carb",2.0,18.0,20.0,NaN,0.0,-2
1,120010000300,WFORMATIONS,fire clay at,0.0,20.0,20.0,NaN,0.0,-2
2,120010000300,WFORMATIONS,interval drift?,18.0,0.0,18.0,NaN,0.0,-2
3,120010000800,WFORMATIONS,coal No. 2 possibly at,0.0,175.0,175.0,NaN,0.0,-2
4,120010000800,WFORMATIONS,interval,175.0,0.0,175.0,NaN,0.0,-2
...,...,...,...,...,...,...,...,...,...
2588272,480590043000,WFORMATIONS,"red clay, firm",120.0,45.0,165.0,NaN,0.0,-2
2588273,480590043000,WFORMATIONS,sand silt & gravel,165.0,6.0,171.0,NaN,0.0,-2
2588274,480590043000,WFORMATIONS,"sandy blue clay, soft",85.0,35.0,120.0,NaN,0.0,-2
2588275,480590043000,WFORMATIONS,stoney blue clay,18.0,67.0,85.0,NaN,0.0,-2


Flags used for target classification purposes:
- -2: No classification 
- -1: Classified, not used/not definitive
- 0: Classified, not target material
- 1: Classified as target material

In [ ]:
downholeData['TARGET'].value_counts()

-2    1337019
-1     722307
0      336977
1      191974
Name: TARGET, dtype: int64

Find all unique wells in downhole dataset

In [ ]:
#Get Unique well APIs
uniqueWells = downholeData['API_NUMBER'].unique()
wellsDF = pd.DataFrame(uniqueWells)
print('Number of unique wells in downholeData: '+str(wellsDF.shape[0]))
wellsDF.columns = ['UNIQUE_API']
wellsDF

Number of unique wells in downholeData: 413902


,UNIQUE_API
0,120010000300
1,120010000800
2,120010000900
3,120010001000
4,120010001100
...,...
413897,480590000900
413898,480590001200
413899,480590027200
413900,480590028100


Sort dataset by API Number and Depth of top of record (will be easier to do data analysis with records in the correct order)

In [ ]:
downholeData_sorted = downholeData.sort_values(['API_NUMBER','TOP'])
downholeData_sorted.reset_index(inplace=True)
downholeData_sorted

,index,API_NUMBER,TABLE_NAME,FORMATION,THICKNESS,TOP,BOTTOM,INTERPRETATION,CLASS_FLAG,TARGET
0,2,120010000300,WFORMATIONS,interval drift?,18.0,0.0,18.0,NaN,0.0,-2
1,0,120010000300,WFORMATIONS,"Colchester No. 2 coal,Penn. carb",2.0,18.0,20.0,NaN,0.0,-2
2,1,120010000300,WFORMATIONS,fire clay at,0.0,20.0,20.0,NaN,0.0,-2
3,4,120010000800,WFORMATIONS,interval,175.0,0.0,175.0,NaN,0.0,-2
4,3,120010000800,WFORMATIONS,coal No. 2 possibly at,0.0,175.0,175.0,NaN,0.0,-2
...,...,...,...,...,...,...,...,...,...,...
2588272,2588273,480590043000,WFORMATIONS,sand silt & gravel,165.0,6.0,171.0,NaN,0.0,-2
2588273,2588276,480590043000,WFORMATIONS,yellow clay,0.0,18.0,18.0,CLAY,1.0,0
2588274,2588274,480590043000,WFORMATIONS,"sandy blue clay, soft",85.0,35.0,120.0,NaN,0.0,-2
2588275,2588272,480590043000,WFORMATIONS,"red clay, firm",120.0,45.0,165.0,NaN,0.0,-2


# Get Bedrock Depth and Layer Thickness

Reproject and align raster grids for surface elevation and bedrock topo (reproject well data too if needed)

In [ ]:
###TO DO 


Use the surface elevation raster and bedrock elevation raster to get depth to bedrock

In [ ]:
###TO DO 

Now, divide by 9 to get thickness of layer at each point in model grid

In [ ]:
##TO DO 

Now, sample each well point (headerData) to get layer thickness, surface elevation, and bedrock 

In [ ]:
#TO DO
#PRevious code that might be useful
#xyzData = pd.read_csv(processDirectory+"headerData__essCols_elev10mDTM.csv",usecols = ['API_NUMBER', 'LONGITUDE','LATITUDE', '10mDTM_ft'])
#downhole_XYZ = pd.merge(downholeData_sorted, xyzData.set_index('API_NUMBER'), on='API_NUMBER',how='left')
#downhole_XYZ['TOP_ELEV_FT'] = downhole_XYZ['10mDTM_ft'] - downhole_XYZ['TOP']
#downhole_XYZ['BOT_ELEV_FT'] = downhole_XYZ['10mDTM_ft'] - downhole_XYZ['BOTTOM']
#downhole_XYZ['TOP_ELEV_M'] = downhole_XYZ['TOP_ELEV_FT'] * 0.3048
#downhole_XYZ['BOT_ELEV_M'] = downhole_XYZ['BOT_ELEV_FT'] * 0.3048
#downhole_XYZ

Calculate  all layer depths/elevations at all wells

In [ ]:
#TO DO

Define function to calculate target thickness in each layer

In [ ]:
##THIS CELL MAY NEED TO BE UPDATED!!!!!


#Define the function to export the result of thickness of target sediments in each layer
def Layers_surfacedown(df, layer = 1):
    
    #Generate Column names based on (looped) integers
    topCol = "ESL_ModelTopoLyrs_"+str(layer)
    if layer != 9: #For all layers except the bottom layer....
        botCol = "ESL_ModelTopoLyrs_"+str(layer+1) #use the layer below it to 
    else: #Otherwise, ...
        botCol = "BedrockCorr" #Use the (corrected) bedrock depth

    #Divide records into 4 separate categories for ease of calculation, to be joined back together later  
        #Category 1: Well interval starts above layer top, ends within model layer
        #Category 2: Well interval is entirely contained withing model layer
        #Category 3: Well interval starts within model layer, continues through bottom of model layer
        #Category 4: well interval begins and ends on either side of model layer (model layer is contained within well layer)

    #records1 = intervals that go through the top of the layer and bottom is within layer
    records1 = df.loc[(df['TOP_ELEV_ft'] > df[topCol]) & (df['BOT_ELEV_ft'] > df[botCol]) & (df['BOT_ELEV_ft'] <= df[topCol]) & (df['BOT_ELEV_ft'] <= df['TOP_ELEV_ft'])].copy()
    records1['TARG_THICK'] = pd.DataFrame(np.round((records1.loc[:,topCol]-records1.loc[: , 'BOT_ELEV_ft']) * records1['Target'],3)).copy()
    
    #records2 = entire interval is within layer
    records2 = df.loc[(df['TOP_ELEV_ft'] <= df[topCol]) & (df['BOT_ELEV_ft'] >= df[botCol]) & (df['BOT_ELEV_ft'] <= df['TOP_ELEV_ft'])].copy()
    records2['TARG_THICK'] = pd.DataFrame(np.round((records2.loc[: , 'TOP_ELEV_ft'] - records2.loc[: , 'BOT_ELEV_ft']) * records2['Target'],3)).copy()

    #records3 = intervals with top within layer and bottom of interval going through bottom of layer
    records3 = df.loc[(df['TOP_ELEV_ft'] > df[botCol]) & (df['BOT_ELEV_ft'] < df[botCol]) & (df['TOP_ELEV_ft'] <= df[topCol]) & (df['BOT_ELEV_ft'] <= df['TOP_ELEV_ft'])].copy()
    records3['TARG_THICK'] = pd.DataFrame(np.round((records3.loc[: , 'TOP_ELEV_ft'] - (records3.loc[:,botCol]))*records3['Target'],3)).copy()

    #records4 = interval goes through entire layer
    records4 = df.loc[(df['TOP_ELEV_ft'] > df[topCol]) & (df['BOT_ELEV_ft'] < df[botCol]) & (df['BOT_ELEV_ft'] <= df['TOP_ELEV_ft'])].copy()
    records4['TARG_THICK'] = pd.DataFrame(np.round((records4.loc[: , topCol]-records4.loc[: , botCol]) * records4['Target'],3)).copy()

    
    #Put the four calculated record categories back together into single dataframe
    res = records1.append(records2).append(records3).append(records4)
    
    res_df = res.groupby(['API_NUMBER','LATITUDE','LONGITUDE'],as_index=False).sum()#calculate thickness for each well interval in the layer indicated (e.g., if there are two well intervals from same well in one model layer)

    res_df['TARG_THICK_PER'] = pd.DataFrame(np.round(res_df['TARG_THICK']/res_df['LyrThick'],3)) #Calculate thickness as percent of total layer thickness
    res_df["LAYER"] = layer #Just to have as part of the output file, include the present layer in the file itself as a separate column
    res_df = res_df[['API_NUMBER', 'LATITUDE', 'LONGITUDE', 'TOP', 'BOTTOM','SURF_ELEV_ft', 'TOP_ELEV_ft', 'BOT_ELEV_ft',topCol,botCol,'LyrThick','TARG_THICK', 'TARG_THICK_PER', 'LAYER']].copy() #Format dataframe for output
    
    return res, res_df

Now run that function over all the layers, looping through each one

In [ ]:
#THIS CELL WILL NEED TO BE UPDATED

outDIR = "\\\\isgs-sinkhole\\geophysics\\Balikian\\ISWS_HydroGeo\\MetroEast_HydroGeo\\CodeOutput\\"+codeTarget+"\\"

for i in np.arange(1,10):
    res, res_df = Layers_surfacedown(df, layer = i)#Run the function defined above for each layer
    outputname = codeTargShort+'Lyr'+str(i)+'.csv' #Create a filename based on the layer and target
    res_df.to_csv(outDIR+outputname)  #Export the file to csv
    #Could also potentially save these to variables for use in following cells

NameError: name 'codeTarget' is not defined

# Interpolate thickness values in each layer

Loop through each layer and interpolate (use same parameters (?))

Ensure rasters align (are co-registered) with grid

# Export

In [ ]:
#Export data 
downhole_bedrockDepth_XYZ.to_csv('\\\\isgs-sinkhole\\geophysics\\Balikian\\BedrockWellData\\Wells\\ProcessedWellData\\Downhole_BedrockPicks.csv',index_label="ID")
wPermits_XYZ.to_csv('\\\\isgs-sinkhole\\geophysics\\Balikian\\BedrockWellData\\Wells\\ProcessedWellData\\wPermits_BedrockPicks.csv',index_label="ID")